# Imports

In [1]:
import src.acquire
import src.prepare
import src.preprocessing

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

# Problem Statement | Background

# Acquire

In this section I'll go over how and where we're getting the data from

## Where the Data is Coming From

This data was provided by the Zillow team as part of a Kaggle competition. More information about the competition can be found here: https://www.kaggle.com/c/zillow-prize-1.

The data from the competition was saved to a mySQL database, which we'll pull into a pandas DataFrame to explore and model off of.

## How the Data is Being Filtered

Here I'll explain what we used to filter information through our SQL query:




# Fill this in!



In [2]:
# Reading in the data from the acquire.py into a dataframe
df = src.acquire.get_zillow_data()

CSV previously generated at `data/raw/zillow_unprocessed.csv`. Reading in that csv as a DataFrame


# Wrangle

## Handling Nulls

### Features Dropped

- What metric are we using to drop features? 
> *If a specific feature is missing more than 50 percent of it's values, we'll drop it*

- What metric are we using to drop rows? 
> *If a row is missing more than 75 percent of it's values, we'll drop it*

In [3]:
# Dropping features and rows with too many missing values
df = src.prepare.handle_missing_values(df)


    Number of rows dropped:    156
    Number of columns dropped: 34
    


> While this helped us get rid of a decent number of unusable observations (rows), this process helps us greatly reduce the number of insignificant features in our data.

### Filling Nulls with Median

> For each feature listed below, we'll be filling the null values with the median of the feature. 

> This in particular will help for these features at it will reduce the effect of outliers within the data.

```
Features
- taxvaluedollarcnt:            $358,880
- taxamount:                    $4448.72
- fullbathcnt:                  2 bathrooms
- lotsizesquarefeet:            7,205 sqft
- calculatedfinishedsquarefeet: 1,542 sqft
- structuretaxvaluedollarcnt:   $136,389
- finishedsquarefeet12          1,523 sqft
```

In [4]:
df = src.prepare.fill_nulls_with_median(df)

### Filling Nulls with Mode

> For each feature listed below, we'll be filling the null values with the mode of the feature. 

```
Features
- regionidzip:           97319
- regionidcity:          12447
- yearbuilt:             1955
- landtaxvaluedollarcnt: $21,299
- lotsizesquarefeet:     6,000 sqft   
```

In [5]:
df = src.prepare.fill_nulls_with_mode(df)

## Dropping Columns

Even though we've created a threshold to drop columns based on missing values, there's still some columns we should drop to clean up the dataframe:

```
Columns to Drop
- buildingqualitytypeid  -|
- heatingorsystemtypeid   | --> These features were used to merge in mySQL
- propertylandusetypeid  -|

- propertycountylandusecode: We've already filtered for single unit residents
- propertyzoningdesc:  Irrelevant data
- censustractandblock: Too many missing values
- Unnamed: 0: Remnant of reading from a csv
```

In [6]:
# Dropping columns
df = src.prepare.drop_columns(df)

# Exploration

In [7]:
# Convert fips to county name, use https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt
df.fips = df.fips.replace({6037.0:'Los_Angeles_County', 6059.0:'Orange_County', 6111.0:'Ventura_County'})

## Splitting the Data

In [8]:
train, test, validate = src.preprocessing.split_data(df)

## Scaling

In [9]:
# Converting the selected features to 'object' types to preserve them
for df in [train, test, validate]:
    src.prepare.numeric_to_object(df)

In [10]:
# Scale the data
for df in [train, test, validate]:
    src.preprocessing.scale_data(df)

## Correlation Testing

Make your titles and labels meaningful, make your number formats friendly to the end user.

### Looking at Correlation By County

In [11]:
# Let's chart out a subset of columns for now, so the charts run in a decent amount of time
df = train[["bathroomcnt", "bedroomcnt", "lotsizesquarefeet", "yearbuilt", "calculatedfinishedsquarefeet", "taxamount", "taxvaluedollarcnt", "logerror", "fips", "longitude", "latitude", "regionidzip"]]

In [ ]:
sns.pairplot(df, hue='fips')

In [ ]:
sns.heatmap(df.corr(), cmap='RdYlBu', annot=True, center=0)

A correlation between number of bedrooms and number of bathrooms makes sense, as well as a correlation between longitude and latitude. 
> We can make a feature to combine bathroom and bedroom count

In [ ]:
plt.figure(figsize=(14,8))
sns.scatterplot(x='longitude', y='latitude', hue='fips', data=df)
plt.xticks(ticks=[])
plt.yticks(ticks=[])
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.show()

# Modeling

# Evaluation

# Conclusion